In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

def detect_and_predict_mask(frame, faceNet, maskNet, min_confidence = 0.3):
    
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224), (104.0, 177.0, 123.0))
    faceNet.setInput(blob)
    detections = faceNet.forward()

    faces = []
    locs = []  # Initialize locs here
    preds = []  # Initialize preds here

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > min_confidence:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (128, 128))
            face = img_to_array(face)
            face = preprocess_input(face)

            faces.append(face)
            locs.append((startX, startY, endX, endY))

    if len(faces) > 0:
        resized_faces = [cv2.resize(face, (128, 128)) for face in faces]
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)
    return (locs, preds)

# load our serialized face detector model from disk
prototxtPath = r"/home/udit/Downloads/deploy.prototxt.txt"
weightsPath = r"/home/udit/Downloads/weights.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector model from disk
maskNet = load_model("/home/udit/mask_detector.model")

# initialize the video stream
# initialize the video stream and allow the camera sensor to warm up
print("[INFO] starting video stream...")
video_source = 0  # Explicitly set to the index of /dev/video0
vs = VideoStream(src=video_source).start()
time.sleep(2.0)

# Check if the video stream is actually open and a frame can be read
cap = cv2.VideoCapture(video_source)
if not cap.isOpened():
    print(f"[ERROR] Could not open video stream at source: {video_source}")
    exit()
while True: 
    ret, frame = cap.read()
    if not ret or frame is None:
        break
    
else:
    print(f"[INFO] Video stream opened successfully at source: {video_source}")
    test_frame = vs.read()
    if test_frame is None:
        print(f"[ERROR] Could not read initial frame from source: {video_source}")
        vs.stop()
        exit()
    else:
        print("[INFO] Successfully read initial frame.")
# loop over the frames from the video stream
while True:
    # grab the frame from the threaded video stream
    frame = vs.read()
    if frame is None:
          break
    resized_frame = imutils.resize(frame, width=400)
    (h, w) = resized_frame.shape[:2]
    try:
        blob = cv2.dnn.blobFromImage(resized_frame, 1.0, (300, 300), (104.0, 177.0, 123.0))
        faceNet.setInput(blob)
        detections = faceNet.forward()
        # ... rest of your face detection and mask prediction code ...
    except Exception as e:
        print(f"[ERROR] OpenCV processing error: {e}")
        break

    # resize the frame for potentially better face detection
    resized_frame = imutils.resize(frame, width=400)
    (h, w) = resized_frame.shape[:2]

    # detect faces in the resized frame
    blob = cv2.dnn.blobFromImage(resized_frame, 1.0, (300, 300),
        (104.0, 177.0, 123.0))
    faceNet.setInput(blob)
    detections = faceNet.forward()
    locs = []
    faces = []

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # extract the face ROI from the *resized_frame*
            face = resized_frame[startY:endY, startX:endX]
            if face.any():  # Ensure face is not empty
                face = cv2.resize(face, (128, 128))
                face = img_to_array(face)
                face = preprocess_input(face)
                faces.append(face)
                locs.append((startX, startY, endX, endY))

    if faces:
        faces = np.array(faces)
        preds = maskNet.predict(faces, batch_size=32)

        for (box, pred) in zip(locs, preds):
            (startX, startY, endX, endY) = box
            (mask, withoutMask) = pred

            label = "Mask" if mask > withoutMask else "No Mask"
            confidence_value = max(mask, withoutMask) * 100
            label_text = "{}: {:.2f}%".format(label, confidence_value)
            color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
            

            cv2.putText(resized_frame, label, (startX, startY - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(resized_frame, (startX, startY), (endX, endY), color, 2)
            # ... (your drawing code, remember the 'box' coordinates are relative to the resized_frame) ...
            print(f"Prediction: {label}, Mask Confidence: {mask:.4f}, No Mask Confidence: {withoutMask:.4f}") # Add this line for debugging

    # show the output frame
    cv2.imshow("Frame", resized_frame) # Show the resized frame with detections
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
   

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

2025-04-25 10:56:07.283459: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2025-04-25 10:56:07.312979: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-04-25 10:56:07.313077: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: udit-Nitro-AN515-57
2025-04-25 10:56:07.313085: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: udit-Nitro-AN515-57
2025-04-25 10:56:07.313307: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 550.120.0
2025-04-25 10:56:07.313335: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 550.120.0
2025-04-25 10:56:07.313341: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 550.120.0
2025-04-25 10:56:07.314091: I tensorflow/

[INFO] starting video stream...


[ WARN:0@3.022] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@3.023] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


[ERROR] Could not open video stream at source: 0
Prediction: No Mask, Mask Confidence: 0.0001, No Mask Confidence: 1.0000
Prediction: No Mask, Mask Confidence: 0.0001, No Mask Confidence: 1.0000
Prediction: No Mask, Mask Confidence: 0.0001, No Mask Confidence: 1.0000
Prediction: No Mask, Mask Confidence: 0.0002, No Mask Confidence: 1.0000
Prediction: No Mask, Mask Confidence: 0.0003, No Mask Confidence: 1.0000
Prediction: No Mask, Mask Confidence: 0.0004, No Mask Confidence: 1.0000
Prediction: No Mask, Mask Confidence: 0.0006, No Mask Confidence: 1.0000
Prediction: No Mask, Mask Confidence: 0.0000, No Mask Confidence: 1.0000
Prediction: No Mask, Mask Confidence: 0.0009, No Mask Confidence: 1.0000
Prediction: No Mask, Mask Confidence: 0.0010, No Mask Confidence: 1.0000
Prediction: No Mask, Mask Confidence: 0.0009, No Mask Confidence: 1.0000
Prediction: No Mask, Mask Confidence: 0.0011, No Mask Confidence: 1.0000
Prediction: No Mask, Mask Confidence: 0.0011, No Mask Confidence: 1.0000
Pr

KeyboardInterrupt: 

In [ ]:
import cv2

cap0 = cv2.VideoCapture(0)
if cap0.isOpened():
    print("Successfully opened /dev/video0")
    ret0, frame0 = cap0.read()
    if ret0:
        print("Successfully read a frame from /dev/video0")
        cv2.imshow("Test Frame", frame0)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Could not read a frame from /dev/video0")
    cap0.release()
else:
    print("Could not open /dev/video0")

Successfully opened /dev/video0
Successfully read a frame from /dev/video0


In [1]:
pip install --upgrade jupyter notebook ipykernel

INFO: pip is looking at multiple versions of notebook to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 4.8 MB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 3.8 MB/s eta 0:00:003.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 5.8 MB/s eta 0:00:00MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 6.0 MB/s eta 0:00:00 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
